# Домашнее задание

**Задание:** Написать веб-сервис (или бота), с которой можно разговаривать: пользователь пишет ей реплику, а она отвечает предложением, в котором все слова заменены на какие-то случайные другие слова той же части речи и с теми же грамматическими характеристиками. Предложение-ответ должно быть согласованным.

Например, на фразу "Мама мыла раму" программа может ответить "Девочка пела песню".

Решение нужно разместить на PythonAnywhere.

Для такой программы вам понадобится большой список русских слов:

можно взять список словоформ с сайта НКРЯ - http://ruscorpora.ru/corpora-freq.html
можно взять просто любой большой текст, вытащить из него слова и использовать их.

Из этого списка вам нужен только список разных лемм разных частей речи, и затем нужно будет использовать функции parse и inflect.

https://github.com/nlpub/pymystem3

https://github.com/nlpub/pymystem3/blob/master/pymystem3/mystem.py

https://tech.yandex.ru/mystem/doc/grammemes-values-docpage/

https://pymorphy2.readthedocs.io/en/latest/
https://pymorphy2.readthedocs.io/en/latest/user/grammemes.html



Установка необходимых библиотек.

In [1]:
!pip install pymorphy2

    100% |████████████████████████████████| 51kB 2.0MB/s 
    100% |████████████████████████████████| 7.1MB 3.9MB/s 
  Running setup.py bdist_wheel for docopt ... - done
  Stored in directory: /content/.cache/pip/wheels/9b/04/dd/7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
Successfully built docopt


In [2]:
!pip install pymystem3

  Running setup.py bdist_wheel for pymystem3 ... - \ | / - done
  Stored in directory: /content/.cache/pip/wheels/55/d3/d3/324a9502a055d4c651f7e61113fe60ad2df54ab7244a6dae6e
Successfully built pymystem3


Скачиваем слова из корпуса русского языка.

In [3]:
!wget http://ruscorpora.ru/ngrams/1grams-3.zip

--2018-05-11 20:47:59--  http://ruscorpora.ru/ngrams/1grams-3.zip
Resolving ruscorpora.ru (ruscorpora.ru)... 94.127.67.66
Connecting to ruscorpora.ru (ruscorpora.ru)|94.127.67.66|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5663668 (5.4M) [application/zip]
Saving to: ‘1grams-3.zip’

1grams-3.zip        100%[===================>]   5.40M   528KB/s    in 9.6s    

2018-05-11 20:48:10 (573 KB/s) - ‘1grams-3.zip’ saved [5663668/5663668]



In [4]:
!unzip 1grams-3.zip

Archive:  1grams-3.zip
  inflating: 1grams-3.txt            


In [0]:
with open('1grams-3.txt', 'r') as f:
  content = f.readlines()

In [6]:
content[0:10]

['6829968\tи\n',
 '5190215\tв\n',
 '3164900\tне\n',
 '2729210\tна\n',
 '2064408\tс\n',
 '2064178\tчто\n',
 '1146383\tкак\n',
 '1092177\tя\n',
 '1046292\tк\n',
 '1040711\tон\n']

In [7]:
content[0].split('\t')[-1].strip()

'и'

In [0]:
words = [line.split('\t')[-1].strip() for line in content]

In [9]:
len(words)

1054210

In [10]:
len(set(words))

1054210

# Программа

Импортируем библиотеки.

In [0]:
from pymystem3 import Mystem
import pymorphy2
from pymorphy2 import MorphAnalyzer

Создаем экземпляры морфологических парсеров.

In [12]:
morph = MorphAnalyzer()
m = Mystem()

Installing mystem to /content/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


Пример функций Mystem

In [0]:
text = 'Мама мыла раму'

In [14]:
ana = m.analyze(text)
print(ana)

[{'analysis': [{'lex': 'мама', 'gr': 'S,жен,од=им,ед'}], 'text': 'Мама'}, {'text': ' '}, {'analysis': [{'lex': 'мыть', 'gr': 'V,несов,пе=прош,ед,изъяв,жен'}], 'text': 'мыла'}, {'text': ' '}, {'analysis': [{'lex': 'рама', 'gr': 'S,жен,неод=вин,ед'}], 'text': 'раму'}, {'text': '\n'}]


In [15]:
lemmas = m.lemmatize(text)
print(lemmas)

['мама', ' ', 'мыть', ' ', 'рама', '\n']


Пример функций PyMorphy

In [16]:
tokens = text.split()
for token in tokens:
  print(morph.parse(token))

[Parse(word='мама', tag=OpencorporaTag('NOUN,anim,femn sing,nomn'), normal_form='мама', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'мама', 1907, 0),))]
[Parse(word='мыла', tag=OpencorporaTag('NOUN,inan,neut sing,gent'), normal_form='мыло', score=0.333333, methods_stack=((<DictionaryAnalyzer>, 'мыла', 54, 1),)), Parse(word='мыла', tag=OpencorporaTag('VERB,impf,tran femn,sing,past,indc'), normal_form='мыть', score=0.333333, methods_stack=((<DictionaryAnalyzer>, 'мыла', 1813, 8),)), Parse(word='мыла', tag=OpencorporaTag('NOUN,inan,neut plur,nomn'), normal_form='мыло', score=0.166666, methods_stack=((<DictionaryAnalyzer>, 'мыла', 54, 6),)), Parse(word='мыла', tag=OpencorporaTag('NOUN,inan,neut plur,accs'), normal_form='мыло', score=0.166666, methods_stack=((<DictionaryAnalyzer>, 'мыла', 54, 9),))]
[Parse(word='раму', tag=OpencorporaTag('NOUN,inan,masc,Geox sing,datv'), normal_form='рам', score=0.5, methods_stack=((<DictionaryAnalyzer>, 'раму', 32, 2),)), Parse(word='раму', tag=Openco

In [17]:
for lemma in lemmas:
  print(morph.parse(lemma))

[Parse(word='мама', tag=OpencorporaTag('NOUN,anim,femn sing,nomn'), normal_form='мама', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'мама', 1907, 0),))]
[Parse(word=' ', tag=OpencorporaTag('UNKN'), normal_form=' ', score=1.0, methods_stack=((<UnknAnalyzer>, ' '),))]
[Parse(word='мыть', tag=OpencorporaTag('INFN,impf,tran'), normal_form='мыть', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'мыть', 1813, 0),))]
[Parse(word=' ', tag=OpencorporaTag('UNKN'), normal_form=' ', score=1.0, methods_stack=((<UnknAnalyzer>, ' '),))]
[Parse(word='рама', tag=OpencorporaTag('NOUN,inan,masc,Geox sing,gent'), normal_form='рам', score=0.5, methods_stack=((<DictionaryAnalyzer>, 'рама', 32, 1),)), Parse(word='рама', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='рама', score=0.5, methods_stack=((<DictionaryAnalyzer>, 'рама', 55, 0),))]
[Parse(word='\n', tag=OpencorporaTag('ROMN'), normal_form='\n', score=1.0, methods_stack=((<RomanNumberAnalyzer>, '\n'),))]


Порядок действий:
1. Распарсить входное предложение - получить части речи и остальные грамматические характеристики. (с помощью pymorphy)
2. Выбрать случайную лемму той же части речи для каждого слова.
3. Применить к каждой лемме inflect с входными грамматическими характеристиками.

Получаем теги частеречной разметки Mystem для каждого слова.

In [18]:
pos_tags = []
for an in ana:
  if 'analysis' in an.keys():
    pos_tags.append(an['analysis'][0]['gr'].split(',')[0])
print(pos_tags)

['S', 'V', 'S']


Нам нужно уметь выбирать новое случайное слово заданной части речи. Для этого необходимо соответствие между словами корпуса и частеречными тегами.
Создаем словарь, где ключами будут части речи, а значениями - принадлежещие им слова.

In [20]:
word_pos_dict = {}
num_words = 10000
for i, word in enumerate(words[:num_words]):
  print(i, end = '\r')
  word = m.lemmatize(word)[0]
  ana = m.analyze(word)
  if 'analysis' in ana[0].keys():
    if len(ana[0]['analysis']) > 0:
      pos_tag = ana[0]['analysis'][0]['gr'].split(',')[0]
      if '=' in pos_tag:
        pos_tag = pos_tag.split('=')[0]
      if pos_tag in word_pos_dict.keys():
        word_pos_dict[pos_tag] = word_pos_dict[pos_tag] + [word]
      else:
        word_pos_dict[pos_tag] = [word]

In [21]:
print('Количество слов:', sum([len(x)  for x in word_pos_dict.values()]))

Количество слов: 9813


In [22]:
word_pos_dict.keys()

dict_keys(['CONJ', 'S', 'PART', 'PR', 'SPRO', 'ADVPRO', 'INTJ', 'V', 'ADV', 'APRO', 'NUM', 'A', 'ANUM'])

Фильтруем по длине, чтобы исключить однобуквенные слова, оставшиеся после автоматической обработки инициалов, аббревиатур и т.п.

In [0]:
candidates_pos = {}
for key, val in word_pos_dict.items():
  if key != 'PART' and key != 'PR' and key != 'SPRO':
    candidates_pos[key] = [x for x in set(val) if len(x) > 1]
  else:
    candidates_pos[key] = set(val)

In [24]:
m = Mystem()
for key, val in candidates_pos.items():
  non_changeable = []
  changeable = []
  for x in val:
    an = m.analyze(x)[0]
    non_changeable_current, changeable_current = an['analysis'][0]['gr'].split('=')
    if '|' in changeable_current:
      changeable_current = changeable_current.replace('(', '').replace(')', '').split("|")[0]
    non_changeable.extend(non_changeable_current.split(','))
    changeable.extend(changeable_current.split(','))
  print(key)
  print('изменяемые:', set(changeable))
  print('неизменяемые:', set(non_changeable))

CONJ
изменяемые: {''}
неизменяемые: {'вводн', 'CONJ'}
S
изменяемые: {'', 'пр', 'род', 'мн', 'вин', 'од', 'муж', 'твор', 'ед', 'им'}
неизменяемые: {'фам', 'отч', 'мж', 'reserved', 'мн', 'имя', 'устар', 'неод', 'од', 'сред', 'S', 'муж', 'им', 'гео', 'разг', 'ед', 'сокр', 'жен'}
PART
изменяемые: {''}
неизменяемые: {'вводн', 'PART'}
PR
изменяемые: {''}
неизменяемые: {'PR'}
SPRO
изменяемые: {'', 'ед', 'вин', 'им'}
неизменяемые: {'3-л', '2-л', 'SPRO', 'мн', 'неод', 'од', 'сред', '1-л', 'муж', 'им', 'ед', 'жен'}
ADVPRO
изменяемые: {''}
неизменяемые: {'вводн', 'ADVPRO'}
INTJ
изменяемые: {''}
неизменяемые: {'INTJ'}
V
изменяемые: {'нп', '3-л', 'изъяв', 'несов', 'прош', 'вин', 'страд', 'прич', 'неод', 'сов', 'полн', 'муж', 'инф', 'непрош', 'пе', 'ед'}
неизменяемые: {'нп', 'несов', 'V', 'сов', 'пе'}
ADV
изменяемые: {'', 'срав', 'вводн'}
неизменяемые: {'вводн', 'прдк', 'ADV'}
APRO
изменяемые: {'пр', 'вин', 'неод', 'од', 'полн', 'муж', 'ед', 'жен'}
неизменяемые: {'APRO', 'кр'}
NUM
изменяемые: {'вин'

 Разбиваем существительные также по одушевлённости (од/неод) и роду (муж/жен/cред), используя Mystem. Это нужно для выбора слов не только по частями речи, но и по другим грамматическим характеристикам.
 Отдельно выделяем имена, фамилии, отчества и географические названия.

In [0]:
named_entites = {}
named_entities_labels = ['гео', 'имя', 'фам', 'отч']

In [0]:
def parse_noun(word):
    an = m.analyze(word)[0]
    if 'analysis' in an.keys():
      non_changeable, changeable = an['analysis'][0]['gr'].split('=')
      tags = non_changeable.split(',')
      gender_tag = tags[1]
      if len(tags) > 2:
            anim_tag = tags[2]
      else:
            anim_tag = '-'
    return tags, gender_tag, anim_tag

In [27]:
m = Mystem()
candidates_nouns_grammar = {}
for i, word in enumerate(candidates_pos['S']):
  is_ne = False
  print('Progress: %d / %d' % (i, len(candidates_pos['S'])), end = '\r')
  tags, gender_tag, anim_tag = parse_noun(word)
  for label in named_entities_labels:
      if label in tags:
        named_entites.setdefault(label,[]).append(word)
        is_ne = True
  if not is_ne:
    candidates_nouns_grammar.setdefault((gender_tag, anim_tag),[]).append(word)

In [28]:
print(candidates_nouns_grammar[('жен', 'неод')][:10])

['печать', 'полночь', 'постановка', 'глава', 'стоимость', 'цель', 'молекула', 'команда', 'тарелка', 'трава']


In [29]:
print(candidates_nouns_grammar[('сред', 'неод')][:10])

['новое', 'заключение', 'бытие', 'достоинство', 'соединение', 'движение', 'солнце', 'наблюдение', 'море', 'письмо']


In [30]:
print(candidates_nouns_grammar[('муж', 'неод')][:10])

['нос', 'руль', 'пистолет', 'остров', 'стол', 'год', 'нож', 'пол', 'момент', 'выпуск']


In [31]:
print(candidates_nouns_grammar[('муж', 'од')][:10])

['жених', 'корреспондент', 'чиновник', 'братец', 'барин', 'генерал', 'кот', 'враг', 'журналист', 'купец']


Глаголы разбиваем аналогичным образом по виду и переходности. 

In [0]:
def parse_verb(word):
    an = m.analyze(word)[0]
    if 'analysis' in an.keys():
      tags = an['analysis'][0]['gr']
      non_changeable, changeable = tags.split('=')
      tags = non_changeable.split(',') + changeable.split(',')
      perf_tag, trans_tag = '-', '-'
      for tag in tags:
        if tag in ['несов', 'сов']:
            perf_tag = tag
        if tag in ['пе', 'нп']:
            trans_tag = tag
    return tags, perf_tag, trans_tag

In [33]:
m = Mystem()
candidates_verbs_grammar = {}
for i, word in enumerate(candidates_pos['V']):
  print('Progress: %d / %d' % (i, len(candidates_pos['V'])), end = '\r')
  tags, perf_tag, trans_tag = parse_verb(word)
  candidates_verbs_grammar.setdefault((perf_tag, trans_tag),[]).append(word)

Пример содержания получившегося словаря.

In [34]:
print(candidates_verbs_grammar[('сов', 'пе')][:10])

['посмотреть', 'полюбить', 'подождать', 'послушать', 'поглядеть', 'посоветовать', 'взять', 'увидать', 'подготовить', 'сунуть']


In [35]:
print(candidates_verbs_grammar[('сов', 'нп')][:10])

['помолчать', 'заснуть', 'поинтересоваться', 'суметь', 'глянуть', 'крикнуть', 'прийти', 'выскочить', 'оглянуться', 'взяться']


In [36]:
print(candidates_verbs_grammar[('несов', 'пе')][:10])

['напоминать', 'объяснять', 'высказывать', 'спрашивать', 'заслуживать', 'знать', 'переживать', 'заключать', 'ждать', 'говорить']


In [37]:
print(candidates_verbs_grammar[('несов', 'нп')][:10])

['дышать', 'приходиться', 'подлежать', 'заключаться', 'проникать', 'отдыхать', 'готовиться', 'надеяться', 'решаться', 'обедать']


Разбиваем местоимения по роду, лицу, одушевлённости и числу (если они не указаны, ставим прочерк).

In [0]:
def parse_pronoun(word):
    an = m.analyze(word)[0]
    if 'analysis' in an.keys():
      non_changeable, changeable = an['analysis'][0]['gr'].split('=')
      tags = non_changeable.split(',')
      gender_tag, person_tag, anim_tag, num_tag = '-', '-', '-', '-'
      for tag in tags:
        if tag in ['муж', 'жен', 'сред']:
            gender_tag = tag
        if '-л' in tag:
            person_tag = tag
        if tag in ['од', 'неод']:
            anim_tag = tag
        if tag in ['мн', 'ед']:
            num_tag = tag
    return tags, person_tag, num_tag, gender_tag, anim_tag

In [39]:
candidates_pronouns_grammar = {}
for i, word in enumerate(candidates_pos['SPRO']):
  print('Progress: %d / %d' % (i, len(candidates_pos['SPRO'])), end = '\r')
  tags, person_tag, num_tag, gender_tag, anim_tag = parse_pronoun(word)
  candidates_pronouns_grammar.setdefault((person_tag, num_tag, gender_tag, anim_tag),[]).append(word)

In [0]:
import pickle

In [0]:
pickle.dump(candidates_pos, open('candidates_pos.p', 'wb'))
pickle.dump(candidates_verbs_grammar, open('candidates_verbs_grammar.p', 'wb'))
pickle.dump(candidates_nouns_grammar, open('candidates_nouns_grammar.p', 'wb'))
pickle.dump(candidates_pronouns_grammar, open('candidates_pronouns_grammar.p', 'wb'))
pickle.dump(named_entites, open('named_entites.p', 'wb'))

In [0]:
# candidates_pos = pickle.load(open('candidates_pos.p', 'rb'))

Сейчас у нас в словаре хранятся множества слов, а не списки.

In [0]:
candidates_pos = {k: list(v) for k, v in candidates_pos.items()}

In [0]:
from google.colab import files

In [0]:
files.download('named_entites.p') 

In [0]:
files.download('candidates_pos.p') 
files.download('candidates_verbs_grammar.p') 
files.download('candidates_nouns_grammar.p')
files.download('candidates_pronouns_grammar.p') 

Теперь, когда у нас есть структура данных для каждого типа речи, которая позволяет выбирать по нескольким неизменяемым грамматическим признакам, мы можем выбрать оттуда слово.

In [0]:
import random

Пример случайного выбора существительного.

In [48]:
random_word = random.choice(candidates_pos['S'])
print(random_word)

пленный


Чтобы переключаться между Pymorphy и Mystem, нужно иметь соотношение обозначений граммем. Записываем вручную.

In [0]:
grammems = {"наст": "pres",
"непрош": "pres",
"прош": "past",
"им": "nomn",
"род": "gent",
"дат": "datv",
"вин": "accs",
"твор": "ablt",
"пр": "loct",
"парт": "gen2",
"местн": "loc2",
"зват": "voct",
"ед": "sing",
"мн": "plur",
"деепр": "GRND",
"инф": "INFN",
"прич": "PRT",
"изъяв": "indc",
"пов": "impr",
"притяж": "Poss",
"прев": "Supr",
"срав": "Cmp2",
"муж": "masc",
"жен": "femn",
"сред": "neut",
"несов": "impf",
"сов": "perf",
"действ": "actv",
"страд": "pssv",
"од": "anim",
"неод": "inan",
"пе": "tran",
"нп": "intr",
"1-л": "1per",
"2-л": "2per",
"3-л": "3per"}

Наконец, пишем полную функцию, которая принимает на вход текст, разбивает его на слова и анализирует с помощью Mystem, затем выбирает случайное слово нужной части речи и с нужными характеристиками и, наконец, копирует изменяемые грамматические признаки из слов оригинала в слова нашего сгенерированного предложения.

In [0]:
def choose_random_word(possible_values):
    return random.choice(possible_values)

In [0]:
def mystem2pymorphy(mystem_grammems):
    pymorphy_grammems = [grammems[x] for x in mystem_grammems if x in grammems.keys()]
    if 'кр' in mystem_grammems:
        pymorphy_grammems = [x if x not in ['PRT', 'ADJ'] else x + 'S' for x in pymorphy_grammems]
    if 'полн' in mystem_grammems:
        pymorphy_grammems = [x if x not in ['PRT', 'ADJ'] else x + 'F' for x in pymorphy_grammems]
    return set(pymorphy_grammems)

In [0]:
def generate_response(text, verbose=True):
    # морфологический разбор текста Mystem'ом
    ana = m.analyze(text)

    if verbose:
        print(ana)

    changed_words = []

    for an in ana:
        word = an['text']
        if 'analysis' in an.keys():
            non_changeable, changeable = an['analysis'][0]['gr'].split('=')
            changeable = changeable.replace("(", "").replace(")", "")
            tags = non_changeable.split(',')
            if '|' in changeable:
                changeable = changeable.split("|")[0]
            pos_tag = tags[0]
            grammems_to_change = changeable.split(',')

            if verbose:
                print("POS tag:", pos_tag)

            if pos_tag == 'S':
                is_ne = False
                tags, gender_tag, anim_tag = parse_noun(word)
                for label in named_entities_labels:
                    if label in tags:
                        possible_values = named_entites[label]
                        is_ne = True
                if not is_ne:
                    possible_values = candidates_nouns_grammar[(gender_tag, anim_tag)]
            elif pos_tag == 'V':
                tags, perf_tag, trans_tag = parse_verb(word)
                possible_values = candidates_verbs_grammar[(perf_tag, trans_tag)]
            elif pos_tag == 'SPRO':
                tags, person_tag, num_tag, gender_tag, anim_tag = parse_pronoun(word)
                possible_values = candidates_pronouns_grammar[(person_tag, num_tag, gender_tag, anim_tag )]
            elif pos_tag == 'PR':
                possible_values = [word]
            else:
                possible_values = candidates_pos[pos_tag]

            random_word = choose_random_word(possible_values)
            if verbose:
                print('Random word:', random_word)
                
            parsed = morph.parse(random_word)[0]
            if verbose:
                print('Grammems to change:', grammems_to_change)
            if len(grammems_to_change) > 0:
                try:
                    inflected_word = parsed.inflect(mystem2pymorphy(grammems_to_change)).word
                except Exception as e:
                    random_word = choose_random_word(possible_values)
                    parsed = morph.parse(random_word)[0]
                    if verbose:
                        print('Random word:', random_word)
                    inflected_word = parsed.inflect(mystem2pymorphy(grammems_to_change)).word
            else:
                inflected_word = word
        else:
            inflected_word = word
            
        if word[0].isupper():
            inflected_word = inflected_word.capitalize()
            
        changed_words.append(inflected_word)
        
    response = ''.join(changed_words)
    if verbose:
        print('Response:', response)

    return response